## Deliverable 2. Create a Customer Travel Destinations Map.

In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Juan,AR,-31.5375,-68.5364,91.36,31,0,19.22,clear sky
1,1,Vaini,TO,-21.2000,-175.2000,84.20,79,20,8.05,few clouds
2,2,Hervey Bay,AU,-25.2986,152.8535,78.01,77,4,3.00,clear sky
3,3,Qaanaaq,GL,77.4840,-69.3632,7.41,76,74,4.99,broken clouds
4,4,Waddan,LY,29.1614,16.1390,49.42,60,0,0.87,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,84.20,79,20,8.05,few clouds
2,2,Hervey Bay,AU,-25.2986,152.8535,78.01,77,4,3.00,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,77.58,72,77,10.13,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,77.00,78,90,10.36,light rain
14,14,Carnarvon,AU,-24.8667,113.6333,77.00,100,0,13.80,clear sky
17,17,Butaritari,KI,3.0707,172.7902,81.32,77,39,22.62,scattered clouds
19,19,General Salgado,BR,-20.6483,-50.3606,75.90,78,99,6.89,overcast clouds
21,21,Puerto Ayora,EC,-0.7393,-90.3518,80.60,78,75,4.61,broken clouds
22,22,Khudumelapye,BW,-23.8833,24.7500,77.74,49,37,5.88,scattered clouds
24,24,Ambon,ID,-3.6954,128.1814,79.00,94,40,4.61,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No empty rows

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,84.20,few clouds,-21.2000,-175.2000,
2,Hervey Bay,AU,78.01,clear sky,-25.2986,152.8535,
9,Rikitea,PF,77.58,broken clouds,-23.1203,-134.9692,
10,Kapaa,US,77.00,light rain,22.0752,-159.3190,
14,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,
17,Butaritari,KI,81.32,scattered clouds,3.0707,172.7902,
19,General Salgado,BR,75.90,overcast clouds,-20.6483,-50.3606,
21,Puerto Ayora,EC,80.60,broken clouds,-0.7393,-90.3518,
22,Khudumelapye,BW,77.74,scattered clouds,-23.8833,24.7500,
24,Ambon,ID,79.00,scattered clouds,-3.6954,128.1814,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [8]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,84.20,few clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Hervey Bay,AU,78.01,clear sky,-25.2986,152.8535,Shelly Bay Resort
9,Rikitea,PF,77.58,broken clouds,-23.1203,-134.9692,Pension Maro'i
10,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
14,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,Hospitality Carnarvon
17,Butaritari,KI,81.32,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
19,General Salgado,BR,75.90,overcast clouds,-20.6483,-50.3606,TUTU HOTEL
21,Puerto Ayora,EC,80.60,broken clouds,-0.7393,-90.3518,Castillo Galapagos
22,Khudumelapye,BW,77.74,scattered clouds,-23.8833,24.7500,
24,Ambon,ID,79.00,scattered clouds,-3.6954,128.1814,Swiss-Belhotel Ambon


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,84.20,few clouds,-21.2000,-175.2000,Keleti Beach Resort
2,Hervey Bay,AU,78.01,clear sky,-25.2986,152.8535,Shelly Bay Resort
9,Rikitea,PF,77.58,broken clouds,-23.1203,-134.9692,Pension Maro'i
10,Kapaa,US,77.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
14,Carnarvon,AU,77.00,clear sky,-24.8667,113.6333,Hospitality Carnarvon
...,...,...,...,...,...,...,...
678,West Bay,KY,84.20,few clouds,19.3667,-81.4167,Calypso Cove
680,Gladstone,AU,78.80,light rain,-23.8500,151.2500,Highpoint International
686,Gizo,SB,84.27,scattered clouds,-8.1030,156.8419,Imagination Island
693,Durban,ZA,79.00,clear sky,-29.8579,31.0292,Hilton Durban


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "weather_database/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index= "City ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))